<a href="https://colab.research.google.com/github/abxda/Taller_UAEMEX_2025/blob/main/Taller_UAEMEX_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install -y gdal-bin libgdal-dev python3-gdal proj-bin libproj-dev --no-install-recommends

In [ ]:
!gdalinfo --version

In [ ]:
#REINICIAR SESION

In [ ]:
!pip install condacolab
import condacolab
condacolab.install()

In [ ]:
#REINICIAR SESION

In [ ]:
!conda install -c conda-forge -y numpy joblib rsgislib gdal libgdal-kea geos kealib scikit-learn scikit-image matplotlib pandas geopandas scipy rasterio shapely pip rtree tqdm jupyterlab xarray openpyxl xlsxwriter jupyterlab_code_formatter Pillow tuiview earthengine-api

In [ ]:
try:
    import rsgislib
    import rsgislib.imageutils
    print("¡RSGISLib importado exitosamente usando Conda!")
    # Opcional: verificar la versión de GDAL que Conda instaló (si lo necesitas)
    # from osgeo import gdal
    # print(f"Versión de GDAL (manejada por Conda): {gdal.__version__}")
except ImportError as e:
    print(f"Error al importar RSGISLib: {e}")
    print("La instalación con Conda podría haber fallado o estar incompleta.")

In [ ]:
#https://drive.google.com/file/d/1JTlU6M2Kt2kpkPOsDmmsCVrT7m1RApAX/view?usp=sharing
!gdown --id 1JTlU6M2Kt2kpkPOsDmmsCVrT7m1RApAX

### Explicación del Código: Generación del Grid Vectorial de Referencia con RSGISLib

Siguiendo nuestra estrategia general, uno de los pasos fundamentales es establecer un marco de referencia espacial consistente para nuestro análisis. Necesitamos dividir nuestra área de estudio, definida por la extensión de la imagen geomediana Sentinel-2 (`GM_AGS_2020.tif`), en una cuadrícula regular de celdas de 100x100 metros. Cada celda de esta cuadrícula actuará como una unidad de análisis, permitiéndonos agregar información y características de diferentes fuentes de datos (como las bandas espectrales y las etiquetas de cobertura terrestre) de manera organizada.

En lugar de depender de herramientas externas como QGIS para crear esta cuadrícula, hemos optado por una estrategia utilizando exclusivamente Python, apoyándonos en la potencia de la biblioteca RSGISLib, complementada por `rasterio` para leer los metadatos iniciales. Este enfoque nos asegura un flujo de trabajo autocontenido y programático.

El siguiente bloque de código implementa esta generación del grid siguiendo un proceso de dos etapas, diseñado para trabajar eficientemente con las herramientas de RSGISLib:

1.  **Creación de un Ráster de IDs Intermedio:** Primero, leemos las propiedades espaciales clave (límites y sistema de referencia de coordenadas - CRS) de nuestra imagen `GM_AGS_2020.tif`. Con esta información, calculamos cuántas filas y columnas de celdas de 100x100 metros se necesitan para cubrir completamente dicha extensión. Luego, generamos un array numérico (usando NumPy) donde cada elemento tiene un identificador único (ID) secuencial. Este array se escribe en un archivo ráster temporal en formato KEA (`temp_id_grid.kea`) usando la función `rsgislib.imageutils.create_img_from_array`. Es crucial asegurar que este ráster intermedio esté georreferenciado correctamente, utilizando las coordenadas de la esquina superior izquierda y las resoluciones adecuadas (notablemente, una resolución Y negativa para alinearse con la convención estándar de imágenes donde el origen está arriba a la izquierda) derivadas de la imagen de referencia. Cada píxel de 100m en este archivo KEA ahora contiene un ID único.

2.  **Poligonización del Ráster de IDs:** Con el ráster de IDs listo, utilizamos la función `rsgislib.vectorutils.createvectors.polygonise_raster_to_vec_lyr`. Esta potente herramienta convierte el ráster en un archivo vectorial. Recorre el ráster de IDs y crea un polígono vectorial para cada grupo de píxeles adyacentes que comparten el mismo valor de ID (en nuestro caso, cada "grupo" es un único píxel de 100m con un ID único). El valor del ID de cada píxel se almacena como un atributo en una columna específica (que llamamos 'id') dentro de la tabla de atributos del archivo vectorial resultante.

**Producto Generado por este Bloque:**

Al finalizar la ejecución de este código, se habrá generado un archivo vectorial en formato **GeoPackage** llamado **`grid_100m_aea.gpkg`**. Este archivo contiene una capa llamada `grid_100m` que consiste en una cuadrícula de polígonos, donde cada polígono representa una celda de 100x100 metros dentro del área de estudio de Aguascalientes. Lo más importante es que cada polígono tiene asociado un **identificador único** en su columna de atributos `id`. Este grid vectorial georreferenciado es la pieza clave que nos permitirá, en los pasos siguientes, vincular y agregar información de diferentes capas ráster utilizando las herramientas de RSGISLib. También se elimina el archivo ráster KEA intermedio que se usó para la poligonización, dejando solo el producto vectorial final.

---

In [ ]:
import os
import rasterio
import numpy as np
import math

from rsgislib import imageutils
from rsgislib import TYPE_32UINT
from rsgislib.vectorutils import createvectors

def create_grid_geopackage_with_rsgislib_polygonize(reference_raster_path,
                                                   output_gpkg_path,
                                                   gpkg_layer_name,
                                                   id_raster_temp_path="temp_id_grid.kea",
                                                   grid_size=100):
    print(f"Starting grid generation for GeoPackage: {output_gpkg_path}")

    try:
        print(f"Reading metadata from reference raster: {reference_raster_path}")
        with rasterio.open(reference_raster_path) as src_ref:
            bounds = src_ref.bounds
            crs_wkt_ref = src_ref.crs.to_wkt()
            if src_ref.crs.is_geographic:
                print("Advertencia: El CRS del ráster de referencia es geográfico.")

        minx, miny, maxx, maxy = bounds
        print(f"  Bounds: {bounds}")
        print(f"  CRS (WKT): {crs_wkt_ref[:100]}...")

        out_cols = math.ceil((maxx - minx) / grid_size)
        out_rows = math.ceil((maxy - miny) / grid_size)
        print(f"  Grid dimensions (100m cells): {out_rows} rows, {out_cols} cols")

        if out_cols == 0 or out_rows == 0:
            print("Error: Calculated grid dimensions are zero.")
            return

        id_array_2d = np.arange(1, (out_rows * out_cols) + 1, dtype=np.uint32).reshape((out_rows, out_cols))

        top_left_x = minx
        top_left_y = maxy

        print(f"  Top-Left X for ID raster: {top_left_x}")
        print(f"  Top-Left Y for ID raster: {top_left_y}")
        print(f"  X-resolution for ID raster: {grid_size}")
        # AHORA Y-RESOLUTION SERÁ NEGATIVA
        print(f"  Y-resolution for ID raster (intended): {-grid_size}")


        print(f"Creating intermediate ID raster: {id_raster_temp_path}")
        image_creation_options = []

        imageutils.create_img_from_array(
            data_arr=id_array_2d,
            output_img=id_raster_temp_path,
            tl_x=top_left_x,
            tl_y=top_left_y,
            out_img_res_x=float(grid_size),
            out_img_res_y=-float(grid_size),
            wkt_string=crs_wkt_ref,
            gdalformat='KEA',
            datatype=TYPE_32UINT,
            options=image_creation_options,
            no_data_val=None
        )
        print(f"  ID raster '{id_raster_temp_path}' created successfully.")

        print(f"Polygonizing ID raster to GeoPackage using RSGISLib: {output_gpkg_path}, layer: {gpkg_layer_name}")
        pixel_value_fieldname = 'id'

        createvectors.polygonise_raster_to_vec_lyr(
            out_vec_file=output_gpkg_path,
            out_vec_lyr=gpkg_layer_name,
            out_format='GPKG',
            input_img=id_raster_temp_path,
            img_band=1,
            pxl_val_fieldname=pixel_value_fieldname,
            replace_file=True,
            replace_lyr=True
        )

        print(f"  GeoPackage layer '{gpkg_layer_name}' in '{output_gpkg_path}' created successfully.")

        # Opcional: Eliminar el ráster de IDs intermedio
        try:
            os.remove(id_raster_temp_path)
            aux_file = id_raster_temp_path + ".aux.xml"
            if os.path.exists(aux_file):
                os.remove(aux_file)
            print(f"  Intermediate ID raster '{id_raster_temp_path}' and aux file removed.")
        except OSError as e:
            print(f"  Warning: Could not remove intermediate ID raster or aux file: {e}")

        print("Grid GeoPackage generation complete.")

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()

reference_raster_file = 'GM_AGS_2020.tif'
output_geopackage_file = 'grid_100m_aea.gpkg'
output_layer_name = 'grid_100m'
temp_id_raster = 'temp_id_grid_for_gpkg.kea'
cell_size = 100

if 'PROJ_LIB' not in os.environ:
    conda_proj_data_dir = '/usr/local/share/proj'
    if os.path.isdir(conda_proj_data_dir):
        print(f"PROJ_LIB no estaba configurado. Estableciendo a: {conda_proj_data_dir}")
        os.environ['PROJ_LIB'] = conda_proj_data_dir
    else:
        print(f"Advertencia: PROJ_LIB no está configurado y {conda_proj_data_dir} no se encontró.")

create_grid_geopackage_with_rsgislib_polygonize(
    reference_raster_path=reference_raster_file,
    output_gpkg_path=output_geopackage_file,
    gpkg_layer_name=output_layer_name,
    id_raster_temp_path=temp_id_raster,
    grid_size=cell_size
)

In [ ]:
from google.colab import files
files.download('grid_100m_aea.gpkg')

In [ ]:
!gdown --id 1J03WNpnqH38hWZfAlheHFmvKdDnsoJSN -O 00l.gpkg

In [ ]:
!gdown --id 1K4Cn2-HHDxfvb9YBdFfS8zGUOjbGPK50 -O limite_ags.zip

In [ ]:
!unzip -o limite_ags.zip -d limite_ags

Ahora que tenemos nuestro grid vectorial de referencia (`grid_100m_aea.gpkg`) listo, el siguiente paso es preparar la capa de información que nos dirá qué áreas dentro de Aguascalientes son consideradas 'urbanas' y cuáles 'no urbanas'. Esta capa actuará como nuestras "etiquetas" o "verdad terreno" (ground truth) que utilizaremos más adelante para entrenar nuestro modelo de Machine Learning.

Para generar estas etiquetas, no partiremos de cero, sino que aprovecharemos la información oficial del **Marco Geoestadístico del Censo de Población y Vivienda 2020** del INEGI, específicamente la capa de localidades (`00l.gpkg`) que descargaste. Combinaremos esta información con el límite geográfico del estado de Aguascalientes (`Region_Aguascalientes.shp`).

El siguiente bloque de código Python utiliza las bibliotecas `geopandas` y `rasterio` para llevar a cabo un proceso de geoprocesamiento que nos permitirá crear un mapa ráster de etiquetas, perfectamente alineado con nuestra imagen Sentinel-2.

**Proceso Clave del Código:**

1.  **Carga y Preparación de Datos Vectoriales:** Se cargan tanto el archivo GeoPackage de localidades como el shapefile del límite estatal de Aguascalientes. Un paso importante aquí es asegurar que ambos conjuntos de datos vectoriales compartan el mismo Sistema de Referencia de Coordenadas (CRS) que nuestra imagen Sentinel-2 de referencia (`GM_AGS_2020.tif`). Si sus CRS originales difieren, el código los reproyectará automáticamente para garantizar la coherencia espacial.
2.  **Filtrado Espacial:** Se seleccionan únicamente aquellas localidades del archivo `00l.gpkg` que se encuentran dentro de los límites de Aguascalientes.
3.  **Clasificación Vectorial (Urbano/No Urbano):** Utilizando el campo `AMBITO` presente en los datos del INEGI, el código identifica las geometrías correspondientes a las áreas 'Urbana' y 'Rural' (interpretando 'Rural' como 'Rural Amanzanada', según la estrategia del tutorial para unificar ambas como zonas urbanas). A todas estas geometrías se les asigna un valor de clase `klass = 1`. Posteriormente, se calcula el área restante dentro de Aguascalientes (mediante una operación de diferencia geométrica) que no corresponde a estas zonas urbanas, y a esta área se le asigna el valor de clase `klass = 2` (no urbano).
4.  **Rasterización:** Este es el paso final y fundamental de este bloque. Las geometrías vectoriales clasificadas (polígonos con valor 1 para urbano y 2 para no urbano) se "queman" o convierten en una imagen ráster (un archivo GeoTIFF). Lo crucial es que esta conversión utiliza la imagen `GM_AGS_2020.tif` como plantilla. Esto asegura que el ráster de etiquetas resultante tenga exactamente la misma extensión, resolución (10 metros), alineación de píxeles y sistema de coordenadas que nuestra imagen Sentinel-2. Cada píxel en el ráster de salida tendrá el valor 1 si corresponde a un área urbana, 2 si corresponde a un área no urbana, o un valor NoData (0 en este caso) si está fuera de las áreas definidas.

**Producto Generado por este Bloque:**

El resultado de ejecutar este código será un nuevo archivo de imagen ráster GeoTIFF llamado **`Etiquetas_1_urbano_2_no_urbano.tif`**. Este archivo es esencial, ya que contiene la clasificación espacial de referencia (urbano vs. no urbano) a una resolución de 10 metros, perfectamente alineada con nuestra imagen Sentinel-2. Será la fuente de verdad que usaremos en los siguientes pasos con RSGISLib para etiquetar nuestras celdas de análisis de 100m.

---

In [ ]:
import geopandas as gpd
import rasterio
from rasterio import features
import numpy as np
from shapely.ops import unary_union
import os
import pandas as pd # Asegurarse que pandas está importado para pd.concat

def generar_raster_etiquetas(
    path_localidades_gpkg,
    path_limite_ags,
    path_raster_referencia,
    path_salida_raster_etiquetas
):
    try:
        print("Proceso iniciado: Generación de Etiquetas_1_urbano_2_no_urbano.tif")

        print(f"Cargando ráster de referencia: {path_raster_referencia}")
        with rasterio.open(path_raster_referencia) as src_ref:
            crs_referencia = src_ref.crs
            transform_referencia = src_ref.transform
            width_referencia = src_ref.width
            height_referencia = src_ref.height
            meta_referencia = src_ref.meta.copy()

        print(f"  CRS de referencia: {crs_referencia}")
        print(f"  Dimensiones de referencia (ancho x alto): {width_referencia} x {height_referencia}")

        print(f"Cargando límite de Aguascalientes: {path_limite_ags}")
        gdf_limite_ags = gpd.read_file(path_limite_ags)
        if gdf_limite_ags.crs != crs_referencia:
            print(f"  Reproyectando límite de Aguascalientes a {crs_referencia}...")
            gdf_limite_ags = gdf_limite_ags.to_crs(crs_referencia)

        # geom_limite_ags_shapely ES el objeto Polygon/MultiPolygon de Shapely
        geom_limite_ags_shapely = unary_union(gdf_limite_ags.geometry)
        if not geom_limite_ags_shapely.is_valid: # Validar temprano
            geom_limite_ags_shapely = geom_limite_ags_shapely.buffer(0)

        # gdf_limite_ags_for_sjoin ES el GeoDataFrame, necesario para sjoin
        gdf_limite_ags_for_sjoin = gpd.GeoDataFrame(geometry=[geom_limite_ags_shapely], crs=crs_referencia)

        print(f"Cargando localidades desde GeoPackage: {path_localidades_gpkg}")
        gdf_localidades_mex = gpd.read_file(path_localidades_gpkg)
        if gdf_localidades_mex.crs != crs_referencia:
            print(f"  Reproyectando localidades del GeoPackage a {crs_referencia}...")
            gdf_localidades_mex = gdf_localidades_mex.to_crs(crs_referencia)

        print("Filtrando localidades dentro de Aguascalientes...")
        gdf_localidades_mex = gdf_localidades_mex[gdf_localidades_mex.geometry.is_valid]

        gdf_localidades_ags = gpd.sjoin(gdf_localidades_mex, gdf_limite_ags_for_sjoin, how="inner", predicate="intersects")

        if gdf_localidades_ags.empty:
            print("Error: No se encontraron localidades dentro del límite de Aguascalientes.")
            return
        print(f"  Se encontraron {len(gdf_localidades_ags)} segmentos de localidades en Aguascalientes.")

        print("Asignando clase 'urbana' (klass=1)...")
        if 'AMBITO' not in gdf_localidades_ags.columns:
            print("Error: El campo 'AMBITO' no se encuentra en el GeoPackage de localidades.")
            print("Por favor, verifica las columnas disponibles:", gdf_localidades_ags.columns)
            return

        condicion_urbana = gdf_localidades_ags['AMBITO'].isin(['Urbana', 'Rural'])
        gdf_urbanas = gdf_localidades_ags[condicion_urbana].copy()

        if gdf_urbanas.empty:
            print("Advertencia: No se encontraron geometrías 'Urbana' o 'Rural' (amanzanada) según el campo AMBITO.")
        gdf_urbanas['klass'] = 1

        geoms_urbanas_union = unary_union(gdf_urbanas.geometry) # Esto es un objeto Shapely

        print("Generando geometrías 'no urbanas' (klass=2)...")
        # geom_limite_ags_shapely es la geometría de Shapely del límite de Ags.

        if geoms_urbanas_union.is_empty or (not isinstance(geoms_urbanas_union, (gpd.geoseries.GeoSeries, gpd.geodataframe.GeoDataFrame)) and not geoms_urbanas_union.is_valid): # Check if it's a shapely geom
            print("  No hay geometrías urbanas válidas para hacer la diferencia, las áreas no urbanas serán todo el límite de Ags.")
            # No es necesario .buffer(0) aquí si ya se validó geom_limite_ags_shapely arriba
            geom_no_urbanas = geom_limite_ags_shapely
        else:
            # Asegurar que geoms_urbanas_union también sea válido si no está vacío
            if not geoms_urbanas_union.is_valid:
                geoms_urbanas_union = geoms_urbanas_union.buffer(0)
            geom_no_urbanas = geom_limite_ags_shapely.difference(geoms_urbanas_union)

        gdf_no_urbanas = gpd.GeoDataFrame(geometry=[geom_no_urbanas], crs=crs_referencia)
        gdf_no_urbanas['klass'] = 2

        print("Combinando geometrías urbanas y no urbanas...")
        gdf_urbanas_validas = gdf_urbanas[gdf_urbanas.geometry.is_valid & ~gdf_urbanas.geometry.is_empty][['geometry', 'klass']]
        gdf_no_urbanas_validas = gdf_no_urbanas[gdf_no_urbanas.geometry.is_valid & ~gdf_no_urbanas.geometry.is_empty][['geometry', 'klass']]

        gdf_final_etiquetas = pd.concat([
            gdf_urbanas_validas,
            gdf_no_urbanas_validas
        ], ignore_index=True)

        gdf_final_etiquetas = gdf_final_etiquetas[~gdf_final_etiquetas.geometry.is_empty & gdf_final_etiquetas.geometry.is_valid]

        if gdf_final_etiquetas.empty:
            print("Error: No hay geometrías finales para rasterizar.")
            return

        shapes_para_rasterizar = [(geom, value) for geom, value in zip(gdf_final_etiquetas.geometry, gdf_final_etiquetas['klass'])]

        meta_salida = meta_referencia.copy()
        meta_salida.update({
            "driver": "GTiff", "height": height_referencia, "width": width_referencia,
            "transform": transform_referencia, "crs": crs_referencia,
            "dtype": rasterio.uint8, "count": 1, "nodata": 0
        })

        print(f"Rasterizando a: {path_salida_raster_etiquetas}")
        with rasterio.open(path_salida_raster_etiquetas, 'w', **meta_salida) as out:
            out_arr = features.rasterize(
                shapes=shapes_para_rasterizar,
                out_shape=(height_referencia, width_referencia),
                transform=transform_referencia,
                fill=0,
                all_touched=True,
                dtype=rasterio.uint8
            )
            out.write(out_arr, 1)

        print(f"Archivo '{path_salida_raster_etiquetas}' generado exitosamente.")

    except Exception as e:
        print(f"Ocurrió un error durante la generación del ráster de etiquetas: {e}")
        import traceback
        traceback.print_exc()


path_localidades_inegi_gpkg = '00l.gpkg'
path_limite_ags_shp = 'limite_ags/Region_Aguascalientes.shp' # Actualizado por ti
path_gm_ags_2020_tif = 'GM_AGS_2020.tif'
path_etiquetas_tif_salida = 'Etiquetas_1_urbano_2_no_urbano.tif'

archivos_a_verificar = {
    "Localidades INEGI (GPKG)": path_localidades_inegi_gpkg,
    "Límite Aguascalientes (SHP)": path_limite_ags_shp,
    "Geomediana (referencia TIFF)": path_gm_ags_2020_tif
}
faltan_archivos = False
for nombre, ruta in archivos_a_verificar.items():
    if not os.path.exists(ruta):
        print(f"Error: El archivo de entrada '{nombre}' no se encontró en la ruta: {ruta}")
        faltan_archivos = True

if not faltan_archivos:
    generar_raster_etiquetas(
        path_localidades_gpkg=path_localidades_inegi_gpkg,
        path_limite_ags=path_limite_ags_shp,
        path_raster_referencia=path_gm_ags_2020_tif,
        path_salida_raster_etiquetas=path_etiquetas_tif_salida
    )
else:
    print("No se puede continuar debido a archivos faltantes. Por favor, verifica las rutas y nombres de archivo.")

In [ ]:
files.download('Etiquetas_1_urbano_2_no_urbano.tif')

Ya tenemos nuestro grid vectorial de 100 metros (`grid_100m_aea.gpkg`) y nuestro mapa de etiquetas urbanas/no urbanas (`Etiquetas_1_urbano_2_no_urbano.tif`).

Ahora, necesitamos una forma de vincular eficientemente estos dos elementos con nuestra imagen satelital de resolución media (Sentinel-2). Aquí es donde entra en juego la potencia de RSGISLib y su concepto de Tablas de Atributos Raster (RAT). El siguiente paso consiste en crear un archivo ráster especial, en formato KEA, que servirá como "índice" espacial para nuestras celdas de análisis de 100 metros, pero manteniendo la resolución fina de nuestra imagen Sentinel-2.

**Proceso Clave del Código:**

El código que utiliza la función `rsgislib.vectorutils.createrasters.rasterise_vec_lyr`. Esta función realiza una operación fundamental: toma nuestra capa vectorial de polígonos de 100 metros (`grid_100m_aea.gpkg`, específicamente la capa `grid_100m` que contiene los polígonos y su columna `id`) y la "rasteriza" o "quema" sobre una plantilla definida por nuestra imagen de referencia (`GM_AGS_2020.tif`).

El resultado no es una imagen de 100m de resolución. En cambio, se genera un nuevo archivo ráster (`grid_100m.kea`) que tiene la **misma resolución de 10 metros** y la misma alineación espacial que `GM_AGS_2020.tif`. La clave está en el valor de cada píxel de 10 metros en este nuevo archivo KEA: en lugar de un valor espectral, cada píxel de 10m recibe el valor del **ID único** de la celda de 100 metros a la que pertenece (extraído de la columna `id` del GeoPackage).

Piensa en este archivo KEA como un mapa detallado a 10m donde cada píxel sabe a qué "zona" o "clump" de 100 metros pertenece. Esta estructura es increíblemente útil porque permite a las funciones posteriores de RSGISLib agrupar todos los píxeles de 10m que comparten el mismo ID (es decir, que pertenecen a la misma celda de 100m) para realizar cálculos zonales eficientes, como calcular proporciones o estadísticas de banda.

Se elige el formato KEA porque es eficiente y, lo más importante, soporta las Tablas de Atributos Raster (RAT), que poblaremos en los siguientes pasos. El tipo de dato se establece como entero de 32 bits (`TYPE_32INT`) para almacenar adecuadamente los identificadores numéricos.

**Producto Generado por este Bloque:**

Al finalizar la ejecución de este código, se habrá generado un archivo ráster en formato KEA llamado **`grid_100m.kea`**. Este archivo tiene una resolución de 10 metros y está perfectamente alineado con `GM_AGS_2020.tif`. El valor de cada píxel en `grid_100m.kea` corresponde al ID de la celda del grid de 100 metros original a la que pertenece ese píxel. Este archivo es la base sobre la cual construiremos nuestra tabla de características en los pasos siguientes.

In [ ]:
import rsgislib.vectorutils.createrasters
from rsgislib import imageutils # Para obtener el tipo de dato de la imagen de referencia si fuera necesario
from rsgislib import TYPE_32INT # Usaremos entero de 32 bits para los IDs

# --- Entradas ---
# El GeoPackage que creamos en el paso anterior
inputVector_gpkg = 'grid_100m_aea.gpkg'
# El nombre de la capa DENTRO del GeoPackage que contiene el grid
inputVectorLyr_name = 'grid_100m' # Este fue el nombre que le dimos al crear el GPKG
# La imagen de referencia para la extensión y resolución del KEA de salida
# Aunque el KEA será de 100m, la referencia puede ser la de 10m para asegurar alineación.
# El script del tutorial usa la geomediana (10m) como referencia para el grid de 100m.
inputImage_ref = 'GM_AGS_2020.tif'
# El campo en el GPKG que contiene los IDs únicos de cada celda del grid
attribute_column_id = 'id' # Este es el nombre que le dimos al campo ID en el GPKG

# --- Salida ---
outputImage_kea = 'grid_100m.kea'

print(f"Iniciando rasterización de '{inputVector_gpkg}' (capa: '{inputVectorLyr_name}') a '{outputImage_kea}'...")
print(f"Usando '{attribute_column_id}' como columna de atributos para los valores del ráster.")
print(f"Imagen de referencia para alineación: '{inputImage_ref}'")

try:
    # El tutorial rasteriza a la resolución de la imagen de referencia, y luego RSGISLib
    # maneja las estadísticas a nivel de los "clumps" definidos por este KEA.
    # El KEA resultante tendrá la misma resolución que inputImage_ref, pero los valores
    # serán los IDs de las celdas de 100m.
    # Si inputImage_ref es de 10m, el KEA también. Cada pixel de 10m dentro de una celda de 100m
    # tendrá el ID de esa celda de 100m.
    # Luego, al popular el RAT, RSGISLib agrupa por estos IDs.

    rsgislib.vectorutils.createrasters.rasterise_vec_lyr(
        vec_file=inputVector_gpkg,
        vec_lyr=inputVectorLyr_name,
        input_img=inputImage_ref,   # Define la extensión y resolución del KEA de salida
        output_img=outputImage_kea,
        datatype=TYPE_32INT,        # Tipo de dato para los IDs
        gdalformat='KEA',
        att_column=attribute_column_id # Columna con los IDs
    )
    print(f"Archivo '{outputImage_kea}' generado exitosamente.")

except Exception as e:
    print(f"Ocurrió un error durante la rasterización del GeoPackage: {e}")
    import traceback
    traceback.print_exc()

Ahora que hemos creado nuestro archivo `grid_100m.kea`, el cual funciona como un mapa índice de media resolución (10m) donde cada píxel conoce el ID de la zona de 100 metros a la que pertenece, es momento de empezar a añadirle inteligencia. Necesitamos vincular estas zonas de 100 metros con la información de clasificación que preparamos en el paso anterior (el archivo `Etiquetas_1_urbano_2_no_urbano.tif`).

**Propósito del Código:**

El siguiente bloque de código utiliza una función clave de RSGISLib, `populate_rat_with_cat_proportions`, para lograr precisamente esto. Su objetivo es analizar cada una de las zonas (o "clumps") de 100 metros definidas en `grid_100m.kea` y determinar qué proporción de su área está cubierta por las clases "urbano" (valor 1) y "no urbano" (valor 2) según nuestro mapa de etiquetas `Etiquetas_1_urbano_2_no_urbano.tif`. Además, identificará cuál de las dos clases es la predominante (la clase mayoritaria) dentro de cada zona de 100 metros.

**Proceso Clave del Código:**

1.  **Entradas:** La función toma como entrada principal nuestro archivo `grid_100m.kea` (el mapa de zonas o `clumps_img`) y el archivo `Etiquetas_1_urbano_2_no_urbano.tif` (el mapa de categorías o `in_cats_img`).
2.  **Cálculo Zonal:** Para cada ID único presente en `grid_100m.kea` (que representa una celda de 100m), la función examina todos los píxeles de 10m correspondientes en `Etiquetas_1_urbano_2_no_urbano.tif`. Cuenta cuántos píxeles pertenecen a la clase 1 y cuántos a la clase 2 dentro de esa zona de 100m.
3.  **Población de la RAT:** Los resultados de estos cálculos no se guardan en un archivo nuevo, sino que se añaden como **nuevas columnas** directamente a la **Tabla de Atributos Raster (RAT)** del archivo `grid_100m.kea`. Específicamente, se añaden:
    * Una columna (llamada `klass` en nuestro caso) que almacena el valor de la clase mayoritaria (1 o 2) para esa zona de 100m.
    * Columnas adicionales (con el prefijo `klass_`, resultando en `klass__1` y `klass__2`) que almacenan la proporción (un valor entre 0.0 y 1.0) del área de la zona cubierta por cada clase respectiva.

**Producto Generado (Modificación):**

Este bloque de código **modifica directamente el archivo `grid_100m.kea`**. Al finalizar su ejecución, la Tabla de Atributos Raster (RAT) dentro de `grid_100m.kea` habrá sido enriquecida con tres nuevas columnas para cada ID de zona (cada fila de la RAT): `klass`, `klass__1`, y `klass__2`. Estas columnas nos proporcionan información cuantitativa sobre la composición de cobertura terrestre (urbana vs. no urbana) dentro de cada una de nuestras celdas de análisis de 100 metros.

---

In [ ]:
from rsgislib import rastergis

# --- Entradas ---
# El KEA generado en el paso anterior (este archivo será MODIFICADO)
clumpsImage_kea = 'grid_100m.kea'
# El ráster de etiquetas que creaste (urbano=1, no-urbano=2)
catsImage_labels = 'Etiquetas_1_urbano_2_no_urbano.tif'

# --- Nombres para las nuevas columnas en la RAT ---
# Prefijo para las columnas de proporción (se crearán klass__1, klass__2)
output_columns_name_prefix = 'klass_' # Usando el nombre del parámetro de la documentación: out_cols_name
# Nombre de la columna para la clase mayoritaria (contendrá 1 o 2)
majority_column_name_klass = 'klass'  # Usando el nombre del parámetro de la documentación: maj_col_name

print(f"Calculando proporciones de clase para '{clumpsImage_kea}' usando '{catsImage_labels}'...")
print(f"  Las nuevas columnas de proporción tendrán el prefijo: '{output_columns_name_prefix}'")
print(f"  La columna de clase mayoritaria se llamará: '{majority_column_name_klass}'")

try:
    # --- CORRECCIÓN DE PARÁMETROS AQUÍ ---
    rastergis.populate_rat_with_cat_proportions(
        in_cats_img=catsImage_labels,       # Nombre de parámetro corregido
        clumps_img=clumpsImage_kea,         # Nombre de parámetro corregido
        out_cols_name=output_columns_name_prefix, # Nombre de parámetro corregido
        maj_col_name=majority_column_name_klass   # Nombre de parámetro corregido
        # Los siguientes son parámetros opcionales con valores por defecto,
        # no los necesitamos para esta tarea básica:
        # cp_cls_names=False,
        # maj_cls_name_field=None, # O el string por defecto si lo hubiera
        # cls_name_field=None,     # O el string por defecto si lo hubiera
        # rat_band_clumps=1,
        # rat_band_cats=1
    )
    print(f"RAT de '{clumpsImage_kea}' actualizada con proporciones y clase mayoritaria.")

except Exception as e:
    print(f"Ocurrió un error al calcular las proporciones: {e}")
    import traceback
    traceback.print_exc()

Ya hemos avanzado bastante. Nuestro archivo `grid_100m.kea` ahora no solo define nuestras zonas de análisis de 100 metros, sino que su Tabla de Atributos Raster (RAT) también contiene información sobre la proporción de cobertura urbana y no urbana dentro de cada una de esas zonas, además de indicar la clase dominante.

El siguiente paso lógico es incorporar la información más rica que poseemos: los datos espectrales directamente de nuestra imagen satelital Sentinel-2 (`GM_AGS_2020.tif`). En lugar de trabajar con cada píxel individual de 10 metros, vamos a resumir las características espectrales dentro de cada una de nuestras celdas de análisis de 100 metros. Calcularemos estadísticas descriptivas para cada banda espectral dentro de cada zona. Estos resúmenes estadísticos se convertirán en las "características" o "features" que nuestro algoritmo de Machine Learning utilizará para aprender a diferenciar entre zonas urbanas y no urbanas.

**Propósito del Código:**

Este bloque de código utiliza la función `rsgislib.rastergis.populate_rat_with_stats` para realizar este cálculo de estadísticas zonales. El objetivo es tomar la imagen geomediana (`GM_AGS_2020.tif`), que contiene los valores de reflectancia para 12 bandas espectrales diferentes, y calcular un conjunto de estadísticas clave (valor mínimo, valor máximo, media, desviación estándar y suma) para cada una de esas 12 bandas, pero agregadas o resumidas para cada zona de 100 metros definida en nuestro archivo `grid_100m.kea`.

**Proceso Clave del Código:**

1.  **Definición de Estadísticas:** Primero, definimos qué estadísticas queremos calcular y cómo queremos que se llamen las columnas resultantes en la RAT. Creamos una lista de objetos `BandAttStats`, uno para cada banda de la 1 a la 12 de la imagen de entrada. Cada objeto especifica el número de banda y los nombres de las columnas para el mínimo (`b{N}Min`), máximo (`b{N}Max`), media (`b{N}Mean`), desviación estándar (`b{N}StdDev`) y suma (`b{N}Sum`) de esa banda.
2.  **Cálculo Zonal:** La función `populate_rat_with_stats` toma la imagen de entrada (`input_img`, nuestra geomediana) y la imagen de zonas o "clumps" (`clumps_img`, nuestro `grid_100m.kea`). Para cada ID único en `clumps_img`, identifica todos los píxeles de 10m correspondientes en `input_img` y calcula las estadísticas definidas (min, max, media, etc.) a partir de los valores de esos píxeles para la banda especificada.
3.  **Población de la RAT:** De manera similar al paso anterior, los resultados (las 5 estadísticas calculadas para cada una de las 12 bandas) se añaden como **nuevas columnas** directamente a la **Tabla de Atributos Raster (RAT)** del archivo `grid_100m.kea`.

**Producto Generado (Modificación):**

Nuevamente, este código **modifica el archivo `grid_100m.kea` existente**. Después de su ejecución, la RAT dentro de este archivo será significativamente más grande, ya que se le habrán añadido 60 nuevas columnas (5 estadísticas * 12 bandas). Cada fila de la RAT (correspondiente a una celda de 100m) contendrá ahora no solo su ID y proporciones de clase, sino también un resumen estadístico detallado de sus características espectrales en todo el espectro capturado por Sentinel-2. Este conjunto completo de atributos es el que finalmente exportaremos para usar en el entrenamiento del modelo.

In [ ]:
from rsgislib import rastergis

# --- Entradas ---
inputImage_geomediana = 'GM_AGS_2020.tif'
clumpsImage_kea = 'grid_100m.kea'

print(f"Calculando estadísticas de banda para '{clumpsImage_kea}' usando '{inputImage_geomediana}'...")

band_stats_definitions = []

for i in range(1, 13):
    band_num = i
    prefix = f'b{band_num}'

    band_stats_definitions.append(
        rastergis.BandAttStats(
            band=band_num,
            min_field=f'{prefix}Min',
            max_field=f'{prefix}Max',
            mean_field=f'{prefix}Mean',
            std_dev_field=f'{prefix}StdDev', # Corregido previamente y correcto
            sum_field=f'{prefix}Sum'
        )
    )
    print(f"  Definidas estadísticas para Banda {band_num}: {prefix}Min, {prefix}Max, {prefix}Mean, {prefix}StdDev (campo: std_dev_field), {prefix}Sum")

try:
    # --- ASEGÚRATE DE QUE ESTA LLAMADA UTILICE LOS NOMBRES DE PARÁMETRO CORRECTOS ---
    rastergis.populate_rat_with_stats(
        input_img=inputImage_geomediana,      # Nombre de parámetro correcto: input_img
        clumps_img=clumpsImage_kea,         # Nombre de parámetro correcto: clumps_img
        band_stats=band_stats_definitions   # Nombre de parámetro correcto: band_stats
        # rat_band es opcional con default = 1, no lo necesitamos aquí.
    )
    print(f"RAT de '{clumpsImage_kea}' actualizada exitosamente con estadísticas de bandas.")

except Exception as e:
    print(f"Ocurrió un error al calcular las estadísticas de banda: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
files.download('grid_100m.kea')

Hemos llegado al punto culminante de nuestra fase de preparación de datos utilizando RSGISLib. A lo largo de los pasos anteriores, hemos construido y enriquecido meticulosamente la Tabla de Atributos Raster (RAT) asociada a nuestro archivo `grid_100m.kea`. Esta tabla ahora contiene, para cada una de nuestras celdas de análisis de 100 metros:

* Su identificador único (ID).
* La clase de cobertura predominante (urbana o no urbana).
* La proporción de área cubierta por cada una de estas clases.
* Un conjunto detallado de estadísticas que resumen las características espectrales de las 12 bandas de la imagen Sentinel-2 dentro de esa celda.

Toda esta información es exactamente lo que necesitamos para entrenar nuestro modelo de Machine Learning. Sin embargo, la RAT dentro del archivo KEA no es un formato directamente consumible por la mayoría de las bibliotecas de ML estándar como Scikit-learn. Por lo tanto, nuestro último paso en esta fase es extraer esta tabla completa a un formato universalmente aceptado: un archivo de texto de Valores Separados por Comas (CSV).

**Proceso Clave del Código:**

El código utiliza la función `rsgislib.rastergis.export_rat_cols_to_ascii` para realizar esta exportación.

1.  **Entradas y Salidas:** Se especifica el archivo KEA (`clumps_img`) que contiene la RAT final y el nombre deseado para el archivo CSV de salida (`out_file`).
2.  **Selección de Campos:** Se define la lista `fields` que contiene los nombres exactos de las columnas de la RAT que deseamos incluir en nuestro archivo CSV. Esta lista abarca la información de clase (`klass`, `klass__1`, `klass__2`) y las 60 columnas de estadísticas espectrales (desde `b1Min` hasta `b12Sum`) que generamos previamente.
3.  **Exportación:** La función `export_rat_cols_to_ascii` lee la RAT del archivo KEA (de la banda especificada, `rat_band=1`), extrae las columnas indicadas en la lista `fields` y las escribe en el archivo CSV de salida. Según la documentación de esta función, la **primera columna** del archivo CSV contendrá automáticamente los identificadores únicos (FID) de las zonas (los valores de los píxeles del archivo `clumps_img`), seguida por las columnas que especificamos en la lista `fields`.

**Producto Generado por este Bloque:**

Este script genera el archivo final de esta fase: **`features_sentinel_urbano.csv`**. Este archivo CSV representa nuestra tabla de características lista para el análisis. Cada fila corresponde a una celda de 100x100 metros, y las columnas contienen el FID, la etiqueta de clase (`klass`) y todas las características numéricas (proporciones y estadísticas espectrales) que describen esa celda. Con este archivo en mano, estamos listos para pasar a la emocionante fase de Machine Learning.

---

In [ ]:
from rsgislib import rastergis
from rsgislib import imageutils # Para inspeccionar columnas si fuera necesario

# --- Entradas ---
# El archivo KEA final con todos los atributos en su RAT
input_clumps_image_kea = 'grid_100m.kea' # Coincide con el parámetro 'clumps_img'

# --- Salida ---
output_csv_file = 'features_sentinel_urbano.csv' # Coincide con el parámetro 'out_file'

print(f"Exportando RAT de '{input_clumps_image_kea}' a '{output_csv_file}'...")

# Lista de nombres de columnas de la RAT a exportar (parámetro 'fields')
# Estos nombres deben ser los mismos que se generaron en los pasos 2.2 y 2.3
fields_to_export = ['klass', 'klass__1', 'klass__2']
for i in range(1, 13): # Para 12 bandas
    prefix = f'b{i}'
    fields_to_export.extend([
        f'{prefix}Min',
        f'{prefix}Max',
        f'{prefix}Mean',
        f'{prefix}StdDev', # Nombre usado en Paso 2.3
        f'{prefix}Sum'
    ])

print(f"  Campos de la RAT que se exportarán (además del ID del clump/celda):")
for field_name in fields_to_export:
    print(f"    - {field_name}")

# Opcional: verificar columnas en RAT
# try:
#     print("\n  Verificando columnas disponibles en la RAT...")
#     rat_columns = imageutils.get_rat_column_names(input_clumps_image_kea, band=1)
#     print(f"    Columnas encontradas en la RAT: {rat_columns}")
# except Exception as e_rat:
#     print(f"    Advertencia: No se pudieron leer las columnas de la RAT para verificación: {e_rat}")

try:
    rastergis.export_rat_cols_to_ascii(
        clumps_img=input_clumps_image_kea, # Nombre de parámetro corregido
        out_file=output_csv_file,        # Nombre de parámetro corregido
        fields=fields_to_export,         # Nombre de parámetro corregido
        rat_band=1                       # Nombre de parámetro corregido (el default es 1, pero lo ponemos explícito)
        # Se eliminó el parámetro 'colid=0' ya que no está en la firma proporcionada
    )

    print(f"\nArchivo CSV '{output_csv_file}' generado exitosamente.")
    print("  Verifica el archivo CSV. La primera columna debería contener los IDs de las celdas (FID),")
    print("  seguida por las columnas especificadas en 'fields'.")

except Exception as e:
    print(f"Ocurrió un error al exportar la RAT a CSV: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
#Bajar 2 Archivos
#/content/features_sentinel_urbano.csv
#/content/grid_100m_aea.gpkg

In [ ]:
files.download('features_sentinel_urbano.csv')

In [ ]:
files.download('grid_100m_aea.gpkg')